In [ ]:
import pandas as pd

In [ ]:
file = "data/4import_Database_Austria_March2018.xlsx"

In [ ]:
def clean_coords(cord):
    lat = str(cord).replace(',', '.')
    lat = lat.replace('°', '')
    return float(lat)

In [ ]:
df = pd.read_excel(file)[1:]

In [ ]:
df

In [ ]:
other_period_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title="otherperiods")

In [ ]:
failed = []
for i, row in df.iterrows():
    temp_site,_ = Site.objects.get_or_create(
        identifier=row['*Site ID']
    )
    temp_site.name=row['*Site Name']
    temp_site.cadastral_number=row['Cadastral Number']
    temp_site.plot_number=row['Plot Number']
    temp_site.description = row['Description\n']
    
    # related other_period
    
    if isinstance(row['Other present periods'], str):
        for y in row['Other present periods'].split(';'):
            temp_period, _ = SkosConcept.objects.get_or_create(
                pref_label=y,
                scheme__in=[other_period_scheme]
            )
            temp_site.other_period.add(temp_period)
    
    # related places:
    if isinstance(row['Cadastral Community'], str):
        temp_nation, _ = Place.objects.get_or_create(
                name=row['*Country']
            )
        temp_land, _ = Place.objects.get_or_create(
                name=row['Municipiality/community'],
                part_of=temp_nation
            )
        temp_county, _ = Place.objects.get_or_create(
                name=row['County'], part_of=temp_land
            )
        lat = clean_coords(row['Coordinates N (Lat)'])
        lng = clean_coords(row['Coordinates E (Long)'])
    
        for x in row['Cadastral Community'].split(";"):
            temp_place, _ = Place.objects.get_or_create(
                name=x, part_of=temp_county, lat=lat, lng=lng
            )
            temp_site.cadastral_community.add(temp_place)
            
            try:
                temp_site.save()
            except:
                pass
    if row['Public'] == "yes":
        temp_site.public = True
    try:
        temp_site.save()
    except:
        failed.append(temp_site)

In [ ]:
for i, row in df.iterrows():
    enttype = row['ArchEntType']
    site, _ = Site.objects.get_or_create(
        identifier=row['*SiteID']
    )
    if isinstance(enttype, str):
        vocab, _ = SkosConcept.objects.get_or_create(
            pref_label = enttype
        )
        print(enttype)
        
        temp_ent, _ = ArchEnt.objects.get_or_create(
            identifier=row['*Entity ID'],
            site_id=site
        )
        temp_ent.name=row['Entity Name']
        temp_ent.comment=row['Comment']
        if row['Public'] == "yes":
            temp_ent.public = True
        temp_ent.ent_type = vocab
        temp_ent.save()
        
    else:
        print(site)